# Intro to Intervals 

Current notebooks demonstrates main operations that can be performed over intervals

# Imports

In [1]:
from OSOL.Extremum.Numerical_Objects.Interval import *

from OSOL.Extremum.Tools.Encoders import CustomEncoder as encoder

In [2]:
import json

# Creation of interval

### Via constructor

In [3]:
i1 = Interval(lower_bound=-1.0, upper_bound=1.0)
i2 = Interval(lower_bound=-3.0, upper_bound=5.0)

print('Interval i1: {}'.format(i1))
print('Interval i2: {}'.format(i2))

### From ordinary number

In [4]:
i3 = Interval.from_value(2.0)

print('Interval i3: {}'.format(i3))

### From dictionary

In [5]:
i4 = Interval.from_dict({'lower_bound': 1.0, 'upper_bound': 3.0})

print('Interval i4: {}'.format(i4))

### Serialization

OSOL.Extremum supports interval serialization to JSON (via custom encoder)

In [6]:
j = json.dumps(i1, cls=encoder)

print(j)

In [7]:
i5 = Interval.from_json(json.loads(j))

print(i5)

### Valid Interval Creation

For numerical computation it is not required to have intervals with arbitrary small width (especially during the solution of optimal control tasks).

Method `Interval.create_valid_interval` can be used for this purpose. It checks whether desired interval exceeds allowed value of width that is controlled via `Interval.__MIN_WIDTH` constant.

In [8]:
print('Interval.__MIN_WIDTH: {}'.format(Interval._Interval__MIN_WIDTH))

Interval.__MIN_WIDTH: 1e-07


Intervals created with this procedure will be automatically degenerated to ordinary numbers if their width is less than `Interval.__MIN_WIDTH`

In [9]:
eps = 1e-5

print('Created via constructor: {}'.format(Interval(1 - eps, 1 + eps)))
print('Created via Interval.create_valid_interval procedure: {}'.format(Interval.create_valid_interval(1 - eps, 1 + eps)))

Created via constructor: [0.99999; 1.00001]
Created via Interval.create_valid_interval procedure: [0.99999; 1.00001]


# Interval properties

In [10]:
print('Target interval: %s\n' % i1)

print('Extraction of lower bound: %f' % i1.left)
print('Extraction of upper bound: %f' % i1.right)

Target interval: [-1.0; 1.0]

Extraction of lower bound: -1.000000
Extraction of upper bound: 1.000000


In [11]:
print('Target interval: %s\n' % i2)

print('Extraction of middle point: %f' % i2.middle_point)
print('Extraction of width: %f' % i2.width)
print('Extraction of radius: %f' % i2.radius)

Target interval: [-3.0; 5.0]

Extraction of middle point: 1.000000
Extraction of width: 8.000000
Extraction of radius: 4.000000


# Operations and functions

### (In)Equality and approximate equality

In [12]:
eps = 1e-7

a = Interval(1, 2)
b = Interval(1, 2)
c = Interval(a.left + eps, a.right - eps)

print('%s == %s => %s' % (a, b, a == b))
print('%s == %s => %s' % (a, c, a == c))
print('%s ~= %s => %s' % (a, c, a.approximately_equals_to(c, max_error=1e-3)))

[1; 2] == [1; 2] => True
[1; 2] == [1.0000001; 1.9999999] => False
[1; 2] ~= [1.0000001; 1.9999999] => True


Remember that real number `x` can be treated as a degenerate interval `[x; x]`

In [13]:
1 == Interval(1, 1)

True

In [14]:
Interval(2, 2) != 3

True

### Interval comparison

It should be noted that comparison is performed only by **lower bound**

In [15]:
a = Interval(1, 3)
b = Interval(1, 2)
c = Interval(2, 2)

print('%s < %s => %s' % (c, a, c < a))
print('%s > %s => %s' % (c, a, c > a))
print('%s >= %s => %s' % (c, a, c >= a))
print('%s < %s => %s' % (a, b, a < b))
print('%s > %s => %s' % (a, b, a > b))
print('%s <= %s => %s' % (a, b, a <= b))

[2; 2] < [1; 3] => False
[2; 2] > [1; 3] => True
[2; 2] >= [1; 3] => True
[1; 3] < [1; 2] => False
[1; 3] > [1; 2] => False
[1; 3] <= [1; 2] => True


### Arithmetical operations and elementary functions

In [16]:
print('%s + %s = %s' % (i1, i2, i1 + i2))
print('%s - %s = %s' % (i1, i2, i1 - i2))
print('\n')

print('%s * %s = %s' % (i3, i4, i3 * i4))
print('%s / %s = %s' % (i3, i4, i3 / i4))
print('\n')

print('%s ** 2 = %s' % (i2, i2 ** 2))
print('%s ** 3 = %s' % (i2, i2 ** 3))

[-1.0; 1.0] + [-3.0; 5.0] = [-4.0; 6.0]
[-1.0; 1.0] - [-3.0; 5.0] = [-6.0; 4.0]


[2.0; 2.0] * [1.0; 3.0] = [2.0; 6.0]
[2.0; 2.0] / [1.0; 3.0] = [0.6666666666666666; 2.0]


[-3.0; 5.0] ** 2 = [0.0; 25.0]
[-3.0; 5.0] ** 3 = [-27.0; 125.0]


In [17]:
print('|%s| = %s' % (i1, abs(i1)))
print('|%s| = %s' % (i2, abs(i2)))
print('\n')

print('exp(%s) = %s' % (i2, exp(i2)))
print('sin(%s) = %s' % (i2, sin(i2)))
print('sqrt(%s) = %s' % (i2, sqrt(i2)))

|[-1.0; 1.0]| = [0.0; 1.0]
|[-3.0; 5.0]| = [0.0; 5.0]


exp([-3.0; 5.0]) = [0.049787068367863944; 148.4131591025766]
sin([-3.0; 5.0]) = [-1.0; 1.0]
sqrt([-3.0; 5.0]) = [0.0; 2.23606797749979]


### Special interval operations

### Bisection

Splits interval by its middle point

In [18]:
i1_left, i1_right = i1.bisect()
print('%s => (%s, %s)' % (i1, i1_left, i1_right))

[-1.0; 1.0] => ([-1.0; 0.0], [0.0; 1.0])


Or you can use `split` methods with specified ratios

In [19]:
print('Target interval: {}'.format(i2))

print('Split result: {}'.format(list(map(str, i2.split(ratios=(3, 2, 1, 2))))))

Target interval: [-3.0; 5.0]
Split result: ['[-3.0; 0.0]', '[0.0; 2.0]', '[2.0; 3.0]', '[3.0; 5.0]']
